In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key='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', 
    model="llama-3.1-8b-instant",
)

response = llm.invoke("The first person who invented the thermometer was...")
#response = llm.invoke("The first person to land on moon was... ")

print(response.content)

The first known thermometer was invented by Galen, a Greek physician, around 2nd century AD. However, his thermometer used a sealed container of air or water and observed temperature-related changes.

The first functional thermometer is often credited to Santorio Santorio, an Italian physician, in 1612. He created a water thermometer consisting of a sealed glass tube filled with water that expanded when heated and contracted when cooled.

However, the modern mercury thermometer was invented by Gabriel Fahrenheit in 1724. Fahrenheit developed a temperature scale that is still used in some parts of the world today.

So the answer to who invented the first thermometer depends on what you consider a 'thermometer'.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/accounting-admin-aventura/job/R-56056")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Accounting Admin - Aventura










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Accounting Admin - Aventura',
  'experience': {'required': 'Must be at least 18 years of retail or consumer service experience',
   'type': 'full-time'},
  'skills': ['Proficient knowledge of office practices, procedures, and equipment',
   'Intermediate skills in Microsoft Office products including Word and Excel',
   'Ability to use the Internet/Intranet as a resource for department work activities',
   'Strong customer service skills and the ability to exercise a high degree of professionalism',
   'Strong attention to detail and deadlines'],
  'description': {'jobResponsibilities': ['Administer advanced cashiering tasks including all cash-related opening/closing duties, assembling/verifying cash deposits, counting tills, and ensuring that cashiers have adequate change on hand',
    'Support scheduling through zone chart report generation and identifying/aiding with opportunities in daily associate coverage',
    'Review associate tech functionality and store facilities t

In [11]:
type(json_res)

list

In [12]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [13]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [14]:
job = json_res[0]

In [15]:
links = collection.query(query_texts=job["skills"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}]]

In [17]:
job

{'role': 'Accounting Admin - Aventura',
 'experience': {'required': 'Must be at least 18 years of retail or consumer service experience',
  'type': 'full-time'},
 'skills': ['Proficient knowledge of office practices, procedures, and equipment',
  'Intermediate skills in Microsoft Office products including Word and Excel',
  'Ability to use the Internet/Intranet as a resource for department work activities',
  'Strong customer service skills and the ability to exercise a high degree of professionalism',
  'Strong attention to detail and deadlines'],
 'description': {'jobResponsibilities': ['Administer advanced cashiering tasks including all cash-related opening/closing duties, assembling/verifying cash deposits, counting tills, and ensuring that cashiers have adequate change on hand',
   'Support scheduling through zone chart report generation and identifying/aiding with opportunities in daily associate coverage',
   'Review associate tech functionality and store facilities to help trou

In [18]:
job = json_res[0]
job["skills"]

['Proficient knowledge of office practices, procedures, and equipment',
 'Intermediate skills in Microsoft Office products including Word and Excel',
 'Ability to use the Internet/Intranet as a resource for department work activities',
 'Strong customer service skills and the ability to exercise a high degree of professionalism',
 'Strong attention to detail and deadlines']

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mushrat, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mushrat, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Seamless Business Processes with AtliQ

Dear Hiring Manager,

I came across the job description for the Accounting Admin role at Nike and was impressed by the requirements and responsibilities outlined. As a business development executive at AtliQ, I believe our expertise in AI & Software Consulting can significantly support your team in fulfilling these needs.

AtliQ has a proven track record of empowering enterprises with tailored solutions, resulting in scalability, process optimization, cost reduction, and heightened overall efficiency. I'd like to highlight a few examples from our portfolio that demonstrate our capabilities:

1. **Android Development:** Our team successfully developed a mobile app for a leading e-commerce company, resulting in a 25% increase in sales and a 30% reduction in customer support queries. [View Case Study](https://example.com/android-portfolio)
2. **DevOps Implementation:** We implemented a DevOps framework for a software development comp